In [1]:
!pip install python-docx
!pip install bert-extractive-summarizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184507 sha256=4b7e2819d112283e22bd7783baf2bf5d22a3b2ad0d3704c008c4af6af92fad6d
  Stored in directory: /root/.cache/pip/wheels/f6/6f/b9/d798122a8b55b74ad30b5f52b01482169b445fbb84a11797a6
Successfully built python-docx
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.2.0
    Uninstalling typing_extensions-4.2.0:
      Successfully uninstalled typing_extensions-4.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-io 0.21.0 requires tensorflow-io-gcs-filesystem==0.21.0, which is not installed.
tensorflow 2.6.3 requires absl-py~=0.10, but you have absl-py 1.0.0 which is incompatible.
tensorflow 2.6.3 requir

## IMPORTS

In [2]:
import docx
from summarizer import Summarizer
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest

## Functions to colorise/hightlight text

In [3]:
def colourise(colour, text):
    if colour == "black":
        return "\033[1;30m" + str(text) + "\033[1;m"
    if colour == "red":
        return "\033[1;31m" + str(text) + "\033[1;m"
    if colour == "green":
        return "\033[1;32m" + str(text) + "\033[1;m"
    if colour == "yellow":
        return "\033[1;33m" + str(text) + "\033[1;m"
    if colour == "blue":
        return "\033[1;34m" + str(text) + "\033[1;m"
    if colour == "magenta":
        return "\033[1;35m" + str(text) + "\033[1;m"
    if colour == "cyan":
        return "\033[1;36m" + str(text) + "\033[1;m"
    if colour == "gray":
        return "\033[1;37m" + str(text) + "\033[1;m"
    return str(text)

def highlight(colour, text):
    if colour == "black":
        return "\033[1;40m" + str(text) + "\033[1;m"
    if colour == "red":
        return "\033[1;41m" + str(text) + "\033[1;m"
    if colour == "green":
        return "\033[1;42m" + str(text) + "\033[1;m"
    if colour == "yellow":
        return "\033[1;43m" + str(text) + "\033[1;m"
    if colour == "blue":
        return "\033[1;44m" + str(text) + "\033[1;m"
    if colour == "magenta":
        return "\033[1;45m" + str(text) + "\033[1;m"
    if colour == "cyan":
        return "\033[1;46m" + str(text) + "\033[1;m"
    if colour == "gray":
        return "\033[1;47m" + str(text) + "\033[1;m"
    return str(text)

#print(colourise("red", "Red"))
#print(highlight("yellow", "Highlight: yellow"))

## Using docx library to get text from file

In [4]:
def getText(filename):
    doc = docx.Document(filename)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return ' . '.join(fullText)

In [5]:
text = getText('../input/instaminutes-task-file/ML Input.docx')
text

"Yes Gauri tell me tell me something about you .   . Okay ahh thank you for allowing me to introduce myself. My name is Gauri shirmali. I am from Udaipur Rajasthan. I'm doing computer science engineering from Geetanjali Institute of technical studies. I secured 9.2 c GPA in 10, standard 73.4 Cgp ahh percentage in 12 standard. And currently an engineering my aggregate score is 77 percentage. computer sciences is omnipresent that is it is present in every field and hence I have invested these last 3.5 almost years and a developing my software skills My technical skills include programming in Python, Java, C, c++, databases I am also familiar with web development, Kubernetes, Docker, Ansible, et cetera. So these all are my technical skills. And I’ve  also used this skills to build several projects, which include user management project and attendance management project. I've also participated in smart India hackathons of which ahh in one we also got to the finals. Ahh these are my technic

In [6]:
def generate_final1(text, summ_split):
    final_text = text
    for i in range(len(summ_split)):
        if summ_split[i] in text and summ_split[i].strip()!='':
            final_text = final_text.replace(summ_split[i], highlight('yellow', summ_split[i]))
            
    final_text = '.'.join([x for x in final_text.split('.') if x])
    return final_text

In [7]:
def generate_final2(text, summary):
    final_text = text
    summ_split = summary.split('.')
    for i in range(len(summ_split)):
        if summ_split[i] in text and summ_split[i].strip()!='':
            final_text = final_text.replace(summ_split[i], highlight('yellow', summ_split[i]))
            
    final_text = '.'.join([x for x in final_text.split('.') if x])
    return final_text

## METHOD 1 -- NER


In [8]:
nlp = spacy.load('en_core_web_lg')

In [9]:
text_split = text.split('.')
summary_ner_list = []
for te in text_split:
    doc = nlp(te)
    if len(doc.ents)!=0:
        summary_ner_list += [te]
summary_ner = ' '.join(summary_ner_list)

In [10]:
summary_ner

"Yes Gauri tell me tell me something about you   My name is Gauri shirmali  I am from Udaipur Rajasthan  I'm doing computer science engineering from Geetanjali Institute of technical studies  I secured 9 2 c GPA in 10, standard 73 4 Cgp ahh percentage in 12 standard  And currently an engineering my aggregate score is 77 percentage  computer sciences is omnipresent that is it is present in every field and hence I have invested these last 3 5 almost years and a developing my software skills My technical skills include programming in Python, Java, C, c++, databases I am also familiar with web development, Kubernetes, Docker, Ansible, et cetera  I've also participated in smart India hackathons of which ahh in one we also got to the finals  And currently I am one of the cultural heads of our students club, So I've I have also participated in several donation camps several donation camps as I am a  a member of the Rotary Club Udaipur, I consider myself as a very focused person, I’ve  and I a

In [11]:
print(generate_final1(text, summary_ner_list))

Yes Gauri tell me tell me something about you .   . Okay ahh thank you for allowing me to introduce myself. My name is Gauri shirmali. I am from Udaipur Rajasthan. I'm doing computer science engineering from Geetanjali Institute of technical studies. I secured 9.2 c GPA in 10, standard 73.4 Cgp ahh percentage in 12 standard. And currently an engineering my aggregate score is 77 percentage. computer sciences is omnipresent that is it is present in every field and hence I have invested these last 3.5 almost years and a developing my software skills My technical skills include programming in Python, Java, C, c++, databases I am also familiar with web development, Kubernetes, Docker, Ansible, et cetera. So these all are my technical skills. And I’ve  also used this skills to build several projects, which include user management project and attendance management project. I've also participated in smart India hackathons of which ahh in one we also got to the finals. Ahh these are my technica

## METHOD 2 -- 
* #### Using Spacy with word frequency
* #### Using pretrained Bert-Large Transformer

In [12]:
def summarize(text, per):
    nlp = spacy.load('en_core_web_sm')
    doc= nlp(text)
    tokens=[token.text for token in doc]
    word_frequencies={}
    
    for word in doc:
        wo = word.text
        wo_lower = wo.lower()
        if wo_lower not in list(STOP_WORDS):
            if wo_lower not in punctuation:
                if word.text not in word_frequencies.keys():
                    word_frequencies[wo] = 1
                else:
                    word_frequencies[wo] += 1
                    
    max_frequency=max(word_frequencies.values())
    for word in word_frequencies.keys():
        word_frequencies[word]=word_frequencies[word]/max_frequency
        
    sentence_tokens= [sent for sent in doc.sents]
    sentence_scores = {}
    
    for sent in sentence_tokens:
        for word in sent:
            wo = word.text
            wo_lower = wo.lower()
            if wo_lower in word_frequencies.keys():
                if sent not in sentence_scores.keys():                            
                    sentence_scores[sent]=word_frequencies[wo_lower]
                else:
                    sentence_scores[sent]+=word_frequencies[wo_lower]
                    
    select_length=int(len(sentence_tokens)*per)
    summary=nlargest(select_length, sentence_scores,key=sentence_scores.get)
    final_summary=[word.text for word in summary]
    summary=' '.join(final_summary)
    return summary

In [13]:
summary_spacy = summarize(text, 0.4)
summary_spacy

"sir like I said there is a work life balance and it a it it promotes like gender equality ,there is no like discrimination on the basis of that.  Sir I choose Tcs only because Tcs is like almost hundred and fifty years old company so it it has a very good job security. Sir technically the latest I want to go with latest technical skills, like not obsolete technologies, but the latest technologies. . Gauri nice talking to you , thankyou sir , and  I request kernel rathor sir to take next part of interview Right. . Otherwise all set all right ok .   . For example, if the dice value for if if the dice value or the that block value in which the counter is right now is five so I have assigned five already that five is a ladder, so then it will increase and for example if 12 is a snake then I've already defined that at 12 you need to decrement the value by for example FIFO or whatever. Yes sir sure it is very important ok sir if they feel that you want to come Sure firstly sir ahh tcs is ha

In [14]:
print(generate_final2(text, summary_spacy))

Yes Gauri tell me tell me something about you .   . Okay ahh thank you for allowing me to introduce myself. My name is Gauri shirmali. I am from Udaipur Rajasthan. I'm doing computer science engineering from Geetanjali Institute of technical studies. I secured 9.2 c GPA in 10, standard 73.4 Cgp ahh percentage in 12 standard. And currently an engineering my aggregate score is 77 percentage. computer sciences is omnipresent that is it is present in every field and hence I have invested these last 3.5 almost years and a developing my software skills My technical skills include programming in Python, Java, C, c++, databases I am also familiar with web development, Kubernetes, Docker, Ansible, et cetera. So these all are my technical skills. And I’ve  also used this skills to build several projects, which include user management project and attendance management project. I've also participated in smart India hackathons of which ahh in one we also got to the finals. Ahh these are my technica

In [15]:
model = Summarizer()
summary_bert = model(text, ratio=0.2)

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [16]:
print(generate_final2(text, summary_bert))

Yes Gauri tell me tell me something about you .   . Okay ahh thank you for allowing me to introduce myself. My name is Gauri shirmali. I am from Udaipur Rajasthan. I'm doing computer science engineering from Geetanjali Institute of technical studies. I secured 9.2 c GPA in 10, standard 73.4 Cgp ahh percentage in 12 standard. And currently an engineering my aggregate score is 77 percentage. computer sciences is omnipresent that is it is present in every field and hence I have invested these last 3.5 almost years and a developing my software skills My technical skills include programming in Python, Java, C, c++, databases I am also familiar with web development, Kubernetes, Docker, Ansible, et cetera. So these all are my technical skills. And I’ve  also used this skills to build several projects, which include user management project and attendance management project. I've also participated in smart India hackathons of which ahh in one we also got to the finals. Ahh these are my technica

In [17]:
file1 = open("NER_extraction.txt","w")
file1.write(summary_ner)
file1.close()

file2 = open("SPACY_extraction.txt","w")
file2.write(summary_spacy)
file2.close()

file3 = open("BERT_extraction.txt","w")
file3.write(summary_bert)
file3.close()